<a href="https://colab.research.google.com/github/tsyganok-ks/tabular-playground-apr-2022/blob/main/Tabular_apr_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install catboost

     |████████████████████████████████| 76.6 MB 1.4 MB/s 


In [2]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/tabular-playground-apr-2022/train.csv')

train_labels_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/tabular-playground-apr-2022/train_labels.csv')

test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/tabular-playground-apr-2022/test.csv')

submit_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/tabular-playground-apr-2022/sample_submission.csv')

Mounted at /content/drive


In [ ]:
X = df.copy()
y = X.pop("price")

# Label encoding for categoricals
for colname in X.select_dtypes("object"):
    X[colname], _ = X[colname].factorize()

# All discrete features should now have integer dtypes (double-check this before using MI!)
discrete_features = X.dtypes == int

from sklearn.feature_selection import mutual_info_regression

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

mi_scores = make_mi_scores(X, y, discrete_features)
mi_scores[::3]  # show a few features with their MI scores

In [ ]:
train_labels_df

,sequence,state
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
25963,25963,1
25964,25964,0
25965,25965,1
25966,25966,1


In [ ]:
train_df

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.000000,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.000000,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.000000,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.000000,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.000000,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558075,25967,327,55,-0.282844,-1.217437,-1.666153,0.586726,-0.930698,-0.451010,-0.651184,0.368702,0.4,0.008671,-0.723536,-0.353909,-0.914749
1558076,25967,327,56,0.130603,0.349790,-1.666153,-0.324779,0.775324,-0.332835,0.099271,0.122137,-0.2,0.644509,0.691407,-0.613169,-0.515772
1558077,25967,327,57,-0.579598,0.429622,-1.666153,0.319469,0.308861,0.282723,-0.512750,0.012214,-1.6,-0.424133,0.716855,1.628601,0.928389
1558078,25967,327,58,1.278980,1.711134,-1.522820,0.802655,-0.460541,-0.055348,2.405282,0.043511,1.9,0.283960,-0.914914,0.364198,0.211424


In [3]:
train_df['output'] = train_df.sequence.map(lambda p: train_labels_df.loc[p,'state'])
train_df

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12,output
0,0,47,0,-0.196291,0.112395,1.000000,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492,0
1,0,47,1,-0.447450,0.134454,1.000000,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390,0
2,0,47,2,0.326893,-0.694328,1.000000,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147,0
3,0,47,3,0.523184,0.751050,1.000000,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241,0
4,0,47,4,0.272025,1.074580,1.000000,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558075,25967,327,55,-0.282844,-1.217437,-1.666153,0.586726,-0.930698,-0.451010,-0.651184,0.368702,0.4,0.008671,-0.723536,-0.353909,-0.914749,0
1558076,25967,327,56,0.130603,0.349790,-1.666153,-0.324779,0.775324,-0.332835,0.099271,0.122137,-0.2,0.644509,0.691407,-0.613169,-0.515772,0
1558077,25967,327,57,-0.579598,0.429622,-1.666153,0.319469,0.308861,0.282723,-0.512750,0.012214,-1.6,-0.424133,0.716855,1.628601,0.928389,0
1558078,25967,327,58,1.278980,1.711134,-1.522820,0.802655,-0.460541,-0.055348,2.405282,0.043511,1.9,0.283960,-0.914914,0.364198,0.211424,0


In [11]:
train_labels_df.loc[0,'state']

0

In [ ]:
train_df.sequence.idxmax()

1558020

In [ ]:
train_df.loc[1,'sensor_00']

-0.4474497681607419

In [ ]:
train_df.iloc[1,3]

-0.4474497681607419

In [ ]:
train_df.iloc[:,1]

In [ ]:
train_df.loc[:,'sensor_00']

In [ ]:
submit_df

,sequence,state
0,25968,0
1,25969,0
2,25970,0
3,25971,0
4,25972,0
...,...,...
12213,38181,0
12214,38182,0
12215,38183,0
12216,38184,0


In [8]:
test_df

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,25968,684,0,2.427357,19.639706,1.00000,-1.466372,-1.289973,-4.207928,2.486339,-2.493893,8.0,-1.123555,-1.673048,10.980453,0.419011
1,25968,684,1,-4.950541,-21.747899,1.00000,0.983186,-0.569053,1.845924,-3.887978,1.727481,-2.9,0.395231,-0.882233,-1.871399,-0.008525
2,25968,684,2,1.136012,-10.756303,1.00000,1.016814,0.964157,2.454749,0.312386,1.154198,-5.6,1.114162,1.525273,-11.584362,0.139812
3,25968,684,3,0.806028,6.504202,1.00000,-0.179646,0.969221,-1.035153,-0.457195,0.254962,-2.7,-0.588873,0.608761,-4.241770,-0.462916
4,25968,684,4,1.288253,5.552521,1.00000,-0.493805,-1.036124,-1.126402,2.008197,-0.730534,0.0,0.899566,-1.259615,-0.472222,-0.121483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733075,38185,773,55,0.211747,2.005252,-1.33282,0.695575,-0.161327,-1.193717,0.421676,0.869466,0.0,-1.536850,0.388101,2.205761,-91.610827
733076,38185,773,56,-0.826121,-2.468487,-1.33282,0.381416,0.144745,1.060583,-0.765938,0.288550,0.2,-1.956647,-0.032158,-1.794239,72.414749
733077,38185,773,57,0.755023,1.469538,-1.33282,-1.253097,-0.414802,0.007479,0.907104,-1.556489,0.4,4.341763,0.150273,0.641975,-34.065644
733078,38185,773,58,-0.187017,0.714286,-1.33282,0.077876,1.323245,0.159312,-0.397996,0.306870,0.1,-1.013728,-0.608616,0.317901,65.659420


In [28]:
#catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier



features = ['subject','step','sequence','sensor_00','sensor_01','sensor_02','sensor_03','sensor_04','sensor_05','sensor_06',
            'sensor_07','sensor_08','sensor_09','sensor_10','sensor_11','sensor_12']
X = train_df[features]
y = train_df.output
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

bb_model = CatBoostClassifier(
                              #cat_features = ['customer_id','postal_code'],
                             random_state=1, 
                              iterations=1000,
                              loss_function='Logloss',
                              eval_metric= 'AUC',
                              task_type='GPU',
                              #thread_count=8,
                              learning_rate=0.2
                             )
bb_model.fit(train_X, train_y, eval_set=(val_X, val_y))

0:	test: 0.5807765	best: 0.5807765 (0)	total: 44.4ms	remaining: 44.4s
1:	test: 0.5995484	best: 0.5995484 (1)	total: 89.4ms	remaining: 44.6s
2:	test: 0.6108823	best: 0.6108823 (2)	total: 135ms	remaining: 44.7s
3:	test: 0.6141991	best: 0.6141991 (3)	total: 178ms	remaining: 44.3s
4:	test: 0.6144287	best: 0.6144287 (4)	total: 224ms	remaining: 44.6s
5:	test: 0.6186979	best: 0.6186979 (5)	total: 268ms	remaining: 44.4s
6:	test: 0.6196757	best: 0.6196757 (6)	total: 308ms	remaining: 43.7s
7:	test: 0.6259561	best: 0.6259561 (7)	total: 352ms	remaining: 43.7s
8:	test: 0.6288933	best: 0.6288933 (8)	total: 389ms	remaining: 42.8s
9:	test: 0.6306599	best: 0.6306599 (9)	total: 430ms	remaining: 42.5s
10:	test: 0.6329775	best: 0.6329775 (10)	total: 473ms	remaining: 42.5s
11:	test: 0.6340511	best: 0.6340511 (11)	total: 510ms	remaining: 42s
12:	test: 0.6355225	best: 0.6355225 (12)	total: 555ms	remaining: 42.2s
13:	test: 0.6396897	best: 0.6396897 (13)	total: 591ms	remaining: 41.6s
14:	test: 0.6411312	best: 

In [29]:
y_pred = bb_model.predict(test_df)
y_pred

array([1, 0, 0, ..., 0, 0, 0])

In [30]:
test_df['output'] = y_pred

In [31]:
test_df.output.value_counts()

0    728383
1      4697
Name: output, dtype: int64

In [33]:
#тут же можно сгруппировать по sequence в тестовых данных
submit_df['state'] = submit_df.sequence.map(lambda p: test_df.loc[test_df.sequence[p],'output'])
submit_df

,sequence,state
0,25968,0
1,25969,0
2,25970,0
3,25971,0
4,25972,0
...,...,...
12213,38181,1
12214,38182,1
12215,38183,1
12216,38184,1


In [35]:
submit_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Kaggle/tabular-playground-apr-2022/submit.csv', index=False)